In [1]:
#from tqdm import tqdm_notebook, tqdm
from tqdm.notebook import tqdm
import alpaca_trade_api as tradeapi
#from MCForecastTools import MCSimulation
from dotenv import load_dotenv
from datetime import datetime
from pathlib import Path
from time import sleep
import pandas as pd
import os

In [2]:
# Alternative?  https://stackoverflow.com/questions/48775841/pandas-ema-not-matching-the-stocks-ema
# ta.ema shown at the end of that page.  Is sorting index necessary?
#import pandas_ta as ta
#
#data["EMA20"] = ta.ema(df2["Close"], length=20)

def add_EMA(price, day):
    return price.ewm(span=day).mean()

# function to check for EMA crossing
def check_EMA_crossing(df):
    # condition 1: EMA18 is higher than EMA50 at the last trading day
    cond_1 = df.iloc[-1]['EMA18'] > df.iloc[-1]['EMA50']
    # condition 2: EMA18 is lower than EMA50 the previous day
    cond_2 = df.iloc[-2]['EMA18'] < df.iloc[-2]['EMA50']
    # condition 3: to filter out stocks with less than 50 candles
    cond_3 = len(df.index) > 50 
    # will return True if all 3 conditions are met
    return (cond_1 and cond_2 and cond_3)

In [3]:
load_dotenv()

sp500_tickers_df = pd.read_csv(Path("dataset/symbols.csv"), header=None)
#display(sp500_tickers_df.head(3))
#display(sp500_tickers_df.tail(3))
sp500_tickers = sp500_tickers_df.loc[:,0]
       
# Setting the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv('ALPACA_API_KEY')
alpaca_api_secret_key = os.getenv('ALPACA_SECRET_KEY')

display(type(alpaca_api_key))
display(type(alpaca_api_secret_key))

# Create the Alpaca tradeapi.REST object
alpaca_api = tradeapi.REST(alpaca_api_key, alpaca_api_secret_key, api_version = "v2")

# Set start and end dates of 10 years back from your current date
timeframe="1D"
today=pd.Timestamp.now()
start_date = pd.Timestamp(today - pd.Timedelta(days=365), tz="America/New_York").isoformat()
end_date = pd.Timestamp(today).isoformat()
max_rows=365

str

str

In [4]:
def get_alpaca_price(ticker):
    ticker_df=alpaca_api.get_barset(
    ticker,
    timeframe,
    start=start_date,
    end=end_date,
    limit=max_rows
    ).df
    
    return (ticker_df)

In [5]:
#price_chart_df = get_alpaca_price('DISK')
#price_chart_df.head()
##close_df = price_chart_df.xs('close',level=1,axis=1)
#close_df = pd.DataFrame(columns = ['close'])
#close_df['close'] = price_chart_df['DISK']['close']
#close_df.head()
#display(close_df['close'])

In [ ]:
# for loop is throttled with sleep() to keep alpacas calls under 200/minute.

# Alternative design would be getting all 500 tickers in one call to get_barset(),
# and extracting the each_stock 'close' column in this loop

# Preliminary ticker list after screening based on EMA
screened_list = [] 

stock_list = sp500_tickers
#for each_stock in tqdm_notebook(stock_list):
for each_stock in tqdm(stock_list):
    #print(each_stock)
    # Step 1: get stock price for each stock
    price_chart_df = get_alpaca_price(each_stock)
    price_chart_df.head()
    close_df = pd.DataFrame(columns = ['close'])
    close_df['close'] = price_chart_df[each_stock]['close']

    # Step 2: add technical indicators (in this case EMA)
    close_df['EMA18']=add_EMA(close_df['close'],18)
    close_df['EMA50']=add_EMA(close_df['close'],50)
    close_df['EMA100']=add_EMA(close_df['close'],100)
    # if all 3 conditions are met, add stock into screened list
    if check_EMA_crossing(close_df):
      screened_list.append(each_stock)
    sleep(0.2)

# 10-year history will be pulled for each ticker in then list, then run Monte Carlo, and other ta (beta, sharpes, etc)
# return (screened_list)
print(screened_list)

  0%|          | 0/505 [00:00<?, ?it/s]